In [7]:
from intelisys import Intelisys, safe_json_loads

In [8]:
# Synchronous usage
intelisys = Intelisys()
response = intelisys.chat("Hello, how are you?")
response

2024-08-10 09:51:11 - Intelisys - INFO - Log level set to: DEBUG
2024-08-10 09:51:11 - Intelisys - INFO - Initializing Intelisys instance 'Intelisys' with provider=anthropic, model=None
2024-08-10 09:51:11 - Intelisys - DEBUG - Intelisys initialized with: name=Intelisys, max_history_words=10000, max_words_per_message=None, json_mode=False, stream=False, use_async=False, max_retry=10, temperature=0, max_tokens=None
2024-08-10 09:51:11 - Intelisys - INFO - Chat method called
2024-08-10 09:51:11 - Intelisys - DEBUG - User input: Hello, how are you?...
2024-08-10 09:51:11 - Intelisys - INFO - get_response method called
2024-08-10 09:51:11 - Intelisys - INFO - Adding message with role: user
2024-08-10 09:51:11 - Intelisys - DEBUG - Message content: Hello, how are you?...
2024-08-10 09:51:11 - Intelisys - DEBUG - API call params: {'model': 'claude-3-5-sonnet-20240620', 'messages': [{'role': 'user', 'content': 'Hello, how are you?'}], 'stream': False, 'temperature': 0, 'max_tokens': 8192}
202

"Hello! I'm doing well, thank you for asking. As an AI assistant, I don't have feelings, but I'm functioning properly and ready to help you with any questions or tasks you might have. How can I assist you today?"

In [9]:
safe_json_loads(response)

{'greeting': 'Hello!',
 'status': "I'm doing well, thank you for asking.",
 'self_description': {'type': 'AI assistant',
  'feelings': "I don't have feelings",
  'functionality': "I'm functioning properly"},
 'offer': 'ready to help you with any questions or tasks you might have',
 'question': 'How can I assist you today?'}

In [10]:
# Template usage
response = intelisys.template_chat(
    template="Summarize this in {{words}} words: {{text}}",
    render_data={"words": 10, "text": "After carefully reviewing the provided resume, I don't see any explicit mention of a specific army rank. The document describes various military and civilian roles and responsibilities, but does not state a particular rank like Lieutenant, Captain, Major, etc. \nThe experience described suggests the individual has had significant leadership roles in the U.S. Army and Army Reserve, including positions like: Chief of the Central Team in the Army Reserve Sustainment Command Transportation Officer in the US Army Reserve Deployment Support Command Director of Logistics for Area Support Group - Kuwait Contingency Contracting Officer in Southwest Asia. While these roles imply a relatively senior position, without an explicitly stated rank, I cannot confirm any specific army rank from the information provided. The focus seems to be more on describing job responsibilities and accomplishments rather than military rank progression."}
)
response

'Resume lacks explicit army rank; describes leadership roles without specifying rank.'

In [11]:
intelisys = Intelisys(json_mode=True)

# Set a default template
intelisys.set_default_template("Return JSON, {{ name }}! {{ question }}")

# Use the template in a chat
response = intelisys.template_chat(
    render_data={"name": "Alice", "question": "How are you today?"}
)
print(response)

2024-08-10 09:51:15 - Intelisys - WARNING - json_mode=True is set for provider 'anthropic'. Note that only OpenAI has native JSON mode support. For other providers, we'll attempt to parse the response as JSON using safe_json_loads.


{'name': 'Alice', 'greeting': 'Hello!', 'mood': "I'm doing well, thank you for asking.", 'current_status': 'Ready to assist', 'timestamp': '2023-05-09T12:34:56Z'}


In [12]:
# Image OCR example
intelisys = Intelisys(provider="openrouter", model="google/gemini-pro-vision")  # Make sure to use a model that supports image processing
result = (intelisys
    .image("https://mintlify.s3-us-west-1.amazonaws.com/anthropic/images/how-to-prompt-eng.png")
    .chat("Please provide the complete text in the following image(s).")
)
result

" Develop test cases\n\nEngineer preliminary prompt\n\nTest prompt against cases\n\nRefine prompt\n\nTest against held-out evals\n\nShip polished prompt\n\nDon't forget edge cases!\n\nEVALS!"